In [12]:
import pandas as pd
big_chunked_data = pd.read_csv("combined-LTCUSDT.csv", chunksize=10_000_000, index_col="time")

store = pd.HDFStore("LTCUSDT.h5", mode="w")
for chunk in big_chunked_data:
    store.append("LTCUSDT", chunk)
store.close()

In [14]:
with pd.HDFStore("LTCUSDT.h5") as f:
    print(f.select("LTCUSDT", where=["index > 1665663289000", "index < 1665749689000"]))

                      id  price     qty   quote_qty  is_buyer_maker  \
time                                                                  
1665663292575  254617480  49.97   0.331    16.54007            True   
1665663294007  254617481  49.98   3.043   152.08914           False   
1665663294007  254617482  49.98   5.268   263.29464           False   
1665663294007  254617483  49.98   1.200    59.97600           False   
1665663294007  254617484  49.98   1.695    84.71610           False   
...                  ...    ...     ...         ...             ...   
1665749687675  254701581  51.93   5.379   279.33147            True   
1665749687675  254701582  51.93  29.603  1537.28379            True   
1665749687675  254701583  51.92   3.347   173.77624            True   
1665749687675  254701584  51.92  25.754  1337.14768            True   
1665749687675  254701585  51.92  12.880   668.72960            True   

               is_best_match  
time                          
1665663292575 

In [17]:
import numpy as np
import h5py

In [26]:
df = pd.DataFrame(np.array(h5py.File("LTCUSDT.h5")['time']))

KeyError: "Unable to open object (object 'time' doesn't exist)"

In [31]:
import tables as tb
h5 = tb.open_file('LTCUSDT.h5', 'r')

In [32]:
h5

File(filename=LTCUSDT.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/LTCUSDT (Group) ''
/LTCUSDT/table (Table(30458167,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Int64Col(shape=(1,), dflt=0, pos=1),
  "values_block_1": Float64Col(shape=(3,), dflt=0.0, pos=2),
  "values_block_2": BoolCol(shape=(2,), dflt=False, pos=3)}
  byteorder := 'little'
  chunkshape := (6241,)
  autoindex := True
  colindexes := {
    "index": Index(6, mediumshuffle, zlib(1)).is_csi=False}

In [37]:
h5.root.LTCUSDT.table[:3]

array([(1656633600670, [245102566], [ 53.84   ,   2.029  , 109.24136], [False,  True]),
       (1656633600670, [245102567], [ 53.86   ,   6.319  , 340.34134], [False,  True]),
       (1656633600670, [245102568], [ 53.87   ,   5.305  , 285.78035], [False,  True])],
      dtype=[('index', '<i8'), ('values_block_0', '<i8', (1,)), ('values_block_1', '<f8', (3,)), ('values_block_2', '?', (2,))])

In [38]:
h5.close()

In [39]:
h5 = pd.HDFStore('LTCUSDT.h5', "r")

In [40]:
h5_copy = h5['LTCUSDT']

In [46]:
h5_copy

,id,price,qty,quote_qty,is_buyer_maker,is_best_match
time,,,,,,
1656633600670,245102566,53.84,2.029,109.24136,False,True
1656633600670,245102567,53.86,6.319,340.34134,False,True
1656633600670,245102568,53.87,5.305,285.78035,False,True
1656633600670,245102569,53.87,4.178,225.06886,False,True
1656633600670,245102570,53.88,12.992,700.00896,False,True
...,...,...,...,...,...,...
1677628796563,275560735,93.79,0.349,32.73271,True,True
1677628796563,275560736,93.79,0.273,25.60467,True,True
1677628796563,275560737,93.79,11.125,1043.41375,True,True


In [50]:
h5_copy.index = pd.to_datetime(h5_copy.index, unit="ms")


In [51]:
resample_time = '5T'

price_candles = h5_copy.price.resample(resample_time).agg(["max","min","first","last"])
price_candles.columns = ["High","Low","Open","Close"]
price_candles["Volume"] = h5_copy.qty.resample(resample_time).sum()
price_candles


,High,Low,Open,Close,Volume
time,,,,,
2022-07-01 00:00:00,53.93,53.44,53.84,53.55,5090.411
2022-07-01 00:05:00,54.99,53.41,53.59,54.88,5681.334
2022-07-01 00:10:00,55.00,54.13,54.90,54.30,7519.137
2022-07-01 00:15:00,54.80,54.18,54.28,54.61,14242.711
2022-07-01 00:20:00,54.62,54.29,54.58,54.38,2856.452
...,...,...,...,...,...
2023-02-28 23:35:00,93.92,93.73,93.77,93.81,519.038
2023-02-28 23:40:00,93.80,93.65,93.80,93.67,1584.310
2023-02-28 23:45:00,93.75,93.60,93.67,93.63,1555.967
